# Desarrollo

## Actividad 1

## Preguntas:
* [a](#a)
* [b](#b)
* [c](#c)
* [d](#d)
* [e](#e)
* [f](#f)
* [g](#g)
* [h](#h)
* [i](#i)
* [j](#j)
* [k](#k)
* [l](#l)
* [m](#m)
* [n](#n)
* [o](#o)
* [p](#p)
* [q](#p)
* [r](#p)
* [s](#s)
* [t](#t)
* [u](#u)
* [v](#v)
* [Actividad 2](#Actividad2)

In [41]:
import pandas as pd
ftr = open("train_data.csv", "r",  encoding="ISO-8859-1")
rows = [line.split(" ",1) for line in ftr.readlines()]
df_train = pd.DataFrame(rows, columns=['Sentiment','Text'])
df_train['Sentiment'] = (pd.to_numeric(df_train['Sentiment'])+1)/2 # 0 o 1

fts = open("test_data.csv", "r",  encoding="ISO-8859-1")
rows = [line.split(" ",1) for line in fts.readlines()]
df_test = pd.DataFrame(rows, columns=['Sentiment','Text'])
df_test['Sentiment'] = (pd.to_numeric(df_test['Sentiment'])+1)/2 # 0 o 1

df_train_text = df_train.Text
df_test_text = df_test.Text
labels_train = df_train.Sentiment.values
labels_test = df_test.Sentiment.values

a)<a class="anchor" id="a"></a>

In [13]:
df_train.head()

,Sentiment,Text
0,0.0,"everything's serious , poetic , earnest and --..."
1,0.0,"narratively , trouble every day is a plodding ..."
2,1.0,a truly wonderful tale combined with stunning ...
3,1.0,jason patric and ray liotta make for one splen...
4,0.0,haneke keeps us at arm's length . guided more ...


In [24]:
suma = 0
i = 0
for sentence in df_train['Text']:
    if i == 0:
        minimo = len(sentence)
        maximo = len(sentence)
    else:
        if minimo>len(sentence):
            minimo = len(sentence)
            min_sent = sentence
        if maximo<len(sentence):
            maximo = len(sentence)
            max_sent = sentence
    i+=1
    suma += len(sentence)
avg_len = suma/len(df_train['Text'])

In [27]:
(avg_len, minimo, maximo, min_sent)

(114.70709060213844, 7, 267, 'crummy\n')

In [28]:
df_test.head()

,Sentiment,Text
0,0.0,screenwriter dan schneider and director shawn ...
1,0.0,"home alone goes hollywood , a funny premise un..."
2,1.0,seldom has a movie so closely matched the spir...
3,0.0,"less dizzying than just dizzy , the jaunt is p..."
4,0.0,an ultra-low-budget indie debut that smacks mo...


In [31]:
suma = 0
i = 0
for sentence in df_test['Text']:
    if i == 0:
        minimo = len(sentence)
        maximo = len(sentence)
    else:
        if minimo>len(sentence):
            minimo = len(sentence)
            min_sent = sentence
        if maximo<len(sentence):
            maximo = len(sentence)
            max_sent = sentence
    i+=1
    suma += len(sentence)
avg_len = suma/len(df_train['Text'])

In [32]:
(avg_len, minimo, maximo, min_sent)

(116.4679234665166, 9, 268, 'horrible\n')

Para un mejor analisis preliminar seria de utilidad la eliminacion de las stopwords a la hora de poder determinar los largos maximo, minimo y el promedio del dataset

b)<a class="anchor" id="b"></a>

In [42]:
from sklearn.model_selection import train_test_split
df_train_text, df_val_text, labels_train, labels_val  = train_test_split(df_train, labels_train, test_size=0.2, random_state=0)

In [43]:
df_train_text.shape

(2843, 2)

In [44]:
df_val_text.shape

(711, 2)

c)<a class="anchor" id="c"></a>

In [48]:
import re, time
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize
def base_word(word):
    wordlemmatizer = WordNetLemmatizer()
    return wordlemmatizer.lemmatize(word) 
def word_extractor(text):
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text) #substitute multiple letter by two
    words = ""
    wordtokens = [ base_word(word.lower()) for word in word_tokenize(text) ]
    for word in wordtokens:
        if word not in commonwords: #delete stopwords
            words+=" "+word
    return words
... #try yourself
word_extractor("I love to eat cake")
word_extractor("I love eating cake")
word_extractor("I loved eating the cake")
word_extractor("I do not love eating cake")
word_extractor("I don't love eating cake")
... #try yourself
texts_train = [word_extractor(text) for text in df_train_text]
texts_val = [word_extractor(text) for text in df_val_text]
texts_test = [word_extractor(text) for text in df_test]

La importancia del preprocesamiento en el dominio del lenguaje natural radica en 